In [60]:
import qiskit as qk
import numpy as np
import scipy.linalg as lin
import math
from qiskit.quantum_info import Statevector , Operator , partial_trace , DensityMatrix
from qiskit.circuit.library.standard_gates import HGate , XGate
from qiskit.circuit.library import GlobalPhaseGate
from qiskit.extensions import Initialize , UnitaryGate
from qiskit.providers.aer import AerSimulator
import sys
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator
from qiskit import QuantumCircuit, Aer, transpile
import importlib
import PMR_LCU
importlib.reload(PMR_LCU)  # Reload to reflect updates
from PMR_LCU import *

/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:94: SyntaxWarning: invalid escape sequence '\o'
  """
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:261: SyntaxWarning: invalid escape sequence '\o'
  Input (int) #2 (q_qbits): The quantum register (not the index) of the q qubits
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:290: SyntaxWarning: invalid escape sequence '\o'
  Input (int) #2 (z_qbits_idx): The index of the register of the |z> qubits


### Testing the permutation operators $U_{cP}$

In [54]:
def Uc_P( Circuit , NumOfPermutations , CtrQbitsIndex , TargQbitsIndex , SubIndex , dagger):
    """"
    This is a void function creating a controlled permutation on the circuit

    Input (int) #1 (NumOfPermutations): The number of spins for the system
    Input (int) #2 (CtrQbitsIndex): The index of the register for the controlled qubits
    Input (int) #3 (TargQbitsIndex): The index of the register for the target qubits
    Input (int) #4 (SubIndex): The sub-index (q) for the i_q register to generate the controlled qubit
    Input (bool) #5 (dagger): Speficies whether the gate is hermitian conjugate of U_p or not

    """
    # Assuming SubIndex refers to a specific block of register of size log2(n) qubits
    if SubIndex > 0:
        CtrQbits = Circuit.qregs[CtrQbitsIndex]
        TargQbits = Circuit.qregs[TargQbitsIndex]
        L = int( np.ceil(np.log2(NumOfPermutations)) )
        NQbits = Circuit.num_qubits # Total qubits for the circuit
        
        CtrQbitsSubset = CtrQbits[(SubIndex-1)*L:SubIndex*L]
        # Making the controlled XX gate for the specific number of controlled qubits
        cxxcirc = qk.QuantumCircuit(2 , name='cXX')
        cxxcirc.x( range(2) )
        cxxGate = cxxcirc.to_gate()
        cxxGate = cxxGate.control(L)
        
        if dagger:
            for i in np.arange(NumOfPermutations-1 , 0 , -1):
                ibin = bin(i)[2:]
                ibin = (L-len(ibin))*'0' +ibin 
                ibin = ibin[::-1]
                
                ZeroCtrls = [x for x in range(L) if(int(ibin[x]) == 0)]
                for j in range(len(ZeroCtrls)):
                    Circuit.x( CtrQbitsSubset[ZeroCtrls[j]] )
                Circuit.append( cxxGate , CtrQbitsSubset[:] + TargQbits[i-1:i+1] )
                for j in np.arange(len(ZeroCtrls)-1 , -1 , -1):
                    Circuit.x( CtrQbitsSubset[ZeroCtrls[j]] )
        else:
            for i in np.arange(1, NumOfPermutations+1):
                ibin = bin(i)[2:]
                ibin = (L-len(ibin))*'0' +ibin 
                ibin = ibin[::-1]

                ZeroCtrls = [x for x in range(L) if(int(ibin[x]) == 0)]
                for j in range(len(ZeroCtrls)):
                    Circuit.x( CtrQbitsSubset[ZeroCtrls[j]] )
                # This part needs to be fixed.. We need to 
                Circuit.append( cxxGate , CtrQbitsSubset[:] + TargQbits[i-1:i+1] )
                for j in range(len(ZeroCtrls)):
                    Circuit.x( CtrQbitsSubset[ZeroCtrls[j]] )

In [61]:
K = 2     # only two modes -w and w
number_of_spins = 4
Qmax = 2
M = number_of_spins-1  # number of permutation operators
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + 2 + Qmax

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins , '|z>') # n+1 st is an ancilla for diagonal rotations!
#ancQubit = qk.QuantumRegister(1 , '|anc>')
qQubits = qk.QuantumRegister(Qmax , '|q>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2 
anc_qbits_index = 3
q_qbits_index = 4

# =========================== Building the circuits  U_0(dt) U_od(dt) ... U_0(dt) U_od(dt) =================================== #

#FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , ancQubit , qQubits )
#FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits )

FullCirc = qk.QuantumCircuit( iqQubits , zQubits)


Uc_P( FullCirc , M , 0 , 1 , 1 , False )
Uc_P( FullCirc , M , 0 , 1 , 2 , False )

print( FullCirc.draw() )


                     ┌───┐         ┌───┐                                    
|iq>_0: ────────■────┤ X ├───■─────┤ X ├─────■──────────────────────────────
        ┌───┐   │    ├───┤   │     └───┘     │                              
|iq>_1: ┤ X ├───■────┤ X ├───■───────────────■──────────────────────────────
        └───┘   │    └───┘   │               │    ┌───┐        ┌───┐        
|iq>_2: ────────┼────────────┼───────■───────┼────┤ X ├───■────┤ X ├───■────
        ┌───┐   │            │       │       │    ├───┤   │    └───┘   │    
|iq>_3: ┤ X ├───┼────────────┼───────■───────┼────┤ X ├───■────────────■────
        └───┘┌──┴───┐        │    ┌──┴───┐   │    └───┘   │            │    
 |z>_0: ─────┤0     ├────────┼────┤0     ├───┼────────────┼────────────┼────
             │  cXX │     ┌──┴───┐│  cXX │   │         ┌──┴───┐        │    
 |z>_1: ─────┤1     ├─────┤0     ├┤1     ├───┼─────────┤0     ├────────┼────
             └──────┘     │  cXX │└──────┘┌──┴───┐     │  cXX │     ┌──┴───┐

In [62]:
IqState1 = Statevector.from_label('0001')
IqState2 = Statevector.from_label('1110')
TotalIqState = IqState1 + IqState2
Zstate = Statevector.from_label('0'*number_of_spins)
TotalState = Zstate.tensor(TotalIqState)
FinalState = TotalState.evolve(FullCirc)
FinalStateIndices = [i for i in range(len(FinalState)) if abs(FinalState[i])>1E-6]
#print(f'The total initial state is {TotalState} and the total final state is {FinalStateIndices}')

In [63]:
Params = [1,2,4,5]

[a , b , c, d] = Params

print(f'a is {a}, b is {b}, and d is {d}')

a is 1, b is 2, and d is 5


### Testing the simulations